In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("en openfoodfacts org products.csv")

In [3]:
df.drop(df[df['pnns_groups_1'] == "unknown"].index, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4735 entries, 6971 to 13218
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   code                                     4735 non-null   int64  
 1   product_name                             4475 non-null   object 
 2   quantity                                 3582 non-null   object 
 3   packaging_en                             1798 non-null   object 
 4   brands_en                                4488 non-null   object 
 5   categories_en                            4653 non-null   object 
 6   origins_en                               888 non-null    object 
 7   labels_en                                2494 non-null   object 
 8   countries_en                             4735 non-null   object 
 9   nutriscore_grade                         4650 non-null   object 
 10  pnns_groups_1                            4650 non

In [5]:
df.dropna(subset=['pnns_groups_2'], inplace=True)

In [6]:
df.to_csv("cleaned_data.csv")

In [7]:
'''

df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

df_merged.sample(5)

'''

"\n\ndf_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')\n\ndf_co2.dropna(subset=['category_name'], inplace=True)\n\ndf_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')\n\ndf_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)\n\ndf_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)\n\ndf_merged.sample(5)\n\n"

In [8]:
# df_merged.to_csv("merged_data.csv")

In [9]:
df_main = pd.read_csv("merged_data.csv")
df_update = pd.read_csv("to_categories.csv")
 
df_main["code"] = df_main["code"].astype(str)
df_update["code"] = df_update["code"].astype(str)
 
df_main_updated = df_main.merge(df_update, on='code', how='left')
df_main_updated
 
# Update only where new categories exist
df_main_updated['pnns_groups_2'] = df_main_updated['new_category (pnns_groups_2)'].combine_first(df_main_updated['pnns_groups_2'])
 
# Drop the helper column
df_main_updated.drop(columns=['new_category (pnns_groups_2)'], inplace=True)
 
df_main_updated.to_csv("updated_products.csv", index=False)
 
df_main_updated.sample(3)

,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
1617,1616,6441130169672,Cereal Bar with Strawberries,NaN,NaN,All-inklusive,"Snacks,Sweet snacks,Bars",NaN,NaN,Finland,unknown,Sugary snacks,Biscuits and cakes,"Sugary snacks,Biscuits and cakes",NaN,NaN,Bars,2.50
3597,3595,6414893505407,Vihanneksenmakuinen nuudeli,85 g,Plastic,Rainbow,"Plant-based foods and beverages,Plant-based fo...",Lithuania,NaN,Finland,unknown,Cereals and potatoes,Cereals,"Cereals and potatoes,Cereals",NaN,NaN,Instant noodles,2.65
3861,3859,8711327325739,Laktoositon vanilja kermajäätelö,1 L,"Cardboard,Paperboard",Ingman,"Desserts,Frozen foods,Frozen desserts,Ice crea...",Finland,"Vegetarian,Avainlippu,No lactose",Finland,d,Milk and dairy products,Ice cream,"Milk and dairy products,Ice cream",NaN,NaN,Ice creams,2.60


In [10]:
df_main_updated['pnns_groups_2'].value_counts()

pnns_groups_2
Sweets                              343
One-dish meals                      292
Chocolate products                  238
Bread                               236
Biscuits and cakes                  234
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      187
Breakfast cereals                   184
Protein snacks and drinks           158
Processed meat                      154
Cereals                             152
Ice cream                           132
Sweetened beverages                 121
Alcoholic beverages                  84
Fats                                 78
Poultry                              76
Vegetables                           71
Artificially sweetened beverages     70
Appetizers                           66
Pastries                             64
Canned foods              

In [11]:
df_main_updated.shape

(4652, 18)

In [12]:
df_main_updated.drop_duplicates(subset=['code'], inplace=True)
df_main_updated.shape

(4650, 18)

In [13]:


to_delete = pd.read_csv("to_delete.csv")

df_main_updated['code'] = df_main_updated['code'].astype(str)
to_delete['code'] = to_delete['code'].astype(str)
df_main_updated = df_main_updated[~df_main_updated['code'].isin(to_delete['code'])]

df_main_updated.shape


(4603, 18)

In [14]:
df_main_updated = df_main_updated.drop(columns=['categories_en', 'nutriscore_grade', 'pnns_groups_1'])
df_main_updated



,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,origins_en,labels_en,countries_en,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
0,0,16229916411,"Foco, 100% pure coconut water with mango",500 ml,NaN,"Foco,Thai-agri-foods-co-ltd",NaN,NaN,"Finland,United States",Unsweetened beverages,"Beverages,Unsweetened beverages",NaN,NaN,Coconut waters,0.55
1,1,640295013004,Paprika original Chips,NaN,NaN,Zweifel,NaN,NaN,"Finland,World",Appetizers,"Salty snacks,Appetizers",NaN,NaN,Crisps,2.10
2,2,644504010013,Toblerone,NaN,NaN,NaN,NaN,NaN,"Finland,France,United Kingdom",Chocolate products,"Sugary snacks,Chocolate products",NaN,unknown,Chocolates,2.95
3,3,20043032,Easter Egg Mix,NaN,NaN,Fresh-easy,NaN,NaN,"Finland,United States",Chocolate products,"Composite foods,One-dish meals",NaN,NaN,Soups,2.95
4,4,20246297,Butter Fudge toffee,150 g,NaN,hatherwood,NaN,NaN,"Finland,France",Sweets,"Sugary snacks,Sweets",NaN,unknown,Salted butter caramels,2.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4647,4645,6411402164902,Reilu,500 g,Plastic,Oululainen,NaN,NaN,Finland,Bread,"Cereals and potatoes,Bread",NaN,unknown,Breads,2.50
4648,4646,6411402578525,Imatran riisipiirakka,6 kpl,fi:kartonki-muovi,Fazer,NaN,NaN,Finland,Pastries,"Composite foods,Pizza pies and quiches",NaN,unknown,Quiches,2.95
4649,4647,6420256010860,NaN,120 g,"Plastic,Other-plastics",Cloetta,NaN,NaN,Finland,Sweets,"Sugary snacks,Sweets",NaN,unknown,Bonbons,2.95
4650,4648,6430053410015,Naan Tandoor - Leipä,250 g,fi:muovipussi,Kirkuk-leipomo,NaN,NaN,Finland,Bread,"Cereals and potatoes,Bread",NaN,unknown,Naans,2.50


In [15]:
to_name = pd.read_csv('to_name.csv')
to_name['code'] = to_name['code'].astype(str)
df_temp = df_main_updated.merge(to_name[['code', 'product_name']], on='code', how='left', suffixes=('', '_new'))
df_temp['product_name'] = df_temp['product_name_new'].combine_first(df_temp['product_name'])
df_temp.drop(columns=['product_name_new'], inplace=True)
df_main_updated = df_temp

In [16]:
new_column_order = ["code", "product_name", "brands_en", "quantity", "packaging_en", "origins_en", "labels_en",	"countries_en", "pnns_groups_2", "food_groups_en", "main_category_en", "environmental_score_score", "environmental_score_grade", "carbon-footprint_1kg"]
df_main_updated = df_main_updated[new_column_order]
df_main_updated.head(10)

,code,product_name,brands_en,quantity,packaging_en,origins_en,labels_en,countries_en,pnns_groups_2,food_groups_en,main_category_en,environmental_score_score,environmental_score_grade,carbon-footprint_1kg
0,16229916411,"Foco, 100% pure coconut water with mango","Foco,Thai-agri-foods-co-ltd",500 ml,NaN,NaN,NaN,"Finland,United States",Unsweetened beverages,"Beverages,Unsweetened beverages",Coconut waters,NaN,NaN,0.55
1,640295013004,Paprika original Chips,Zweifel,NaN,NaN,NaN,NaN,"Finland,World",Appetizers,"Salty snacks,Appetizers",Crisps,NaN,NaN,2.10
2,644504010013,Toblerone,NaN,NaN,NaN,NaN,NaN,"Finland,France,United Kingdom",Chocolate products,"Sugary snacks,Chocolate products",Chocolates,NaN,unknown,2.95
3,20043032,Easter Egg Mix,Fresh-easy,NaN,NaN,NaN,NaN,"Finland,United States",Chocolate products,"Composite foods,One-dish meals",Soups,NaN,NaN,2.95
4,20246297,Butter Fudge toffee,hatherwood,150 g,NaN,NaN,NaN,"Finland,France",Sweets,"Sugary snacks,Sweets",Salted butter caramels,NaN,unknown,2.95
5,20575557,Medium egg noodles,Combino,250 g,NaN,NaN,NaN,"Finland,France,United Kingdom",Cereals,"Cereals and potatoes,Cereals",Egg noodles,NaN,unknown,2.65
6,20683191,Cookie brownie,Mcennedy,8 x 30 g,NaN,NaN,NaN,"Finland,France,Spain",Biscuits and cakes,"Sugary snacks,Biscuits and cakes",Biscuits and cakes,NaN,unknown,2.50
7,20886561,Sweet liquorice lakritsijäätelö,Lidl,68 g,NaN,NaN,NaN,"Finland,Norway",Ice cream,"Milk and dairy products,Ice cream",fi:lakritsijäätelöpuikot,NaN,NaN,2.60
8,2244898002793,AH Vloerbrood meergranen,Albert Heijn,NaN,NaN,NaN,NaN,"Finland,Netherlands",Bread,"Cereals and potatoes,Bread",Breads,NaN,NaN,2.50
9,2244899002693,AH Vloerbrood Meergranen HL,Albert Heijn,NaN,NaN,NaN,NaN,"Finland,Netherlands",Bread,"Cereals and potatoes,Bread",Breads,NaN,NaN,2.50


In [17]:
df_main_updated['carbon-footprint_1kg'].isna().sum()

np.int64(0)

In [18]:
df_main_updated['pnns_groups_2'].value_counts()

pnns_groups_2
Sweets                              340
One-dish meals                      292
Chocolate products                  234
Bread                               232
Biscuits and cakes                  229
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      185
Breakfast cereals                   183
Protein snacks and drinks           158
Processed meat                      154
Cereals                             151
Ice cream                           131
Sweetened beverages                 118
Fats                                 78
Alcoholic beverages                  77
Poultry                              76
Vegetables                           70
Artificially sweetened beverages     68
Appetizers                           64
Pastries                             63
Canned foods              

In [19]:
df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v3.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

In [20]:
df_main_updated = df_main_updated.rename(columns={"code": "Barcode", "product_name": "Product name", "brands_en": "Brand", "quantity": "Quantity","packaging_en": "Packaging material", "origins_en": "Origin", "labels_en": "Label", "countries_en": "Sale countries", "pnns_groups_2": "Product group", "food_groups_en": "Food group", "main_category_en": "Product main category", "environmental_score_score": "Environmental score", "environmental_scrore_grade": "Environmental grade", "carbon-footprint_1kg": "Carbon-footprint 1kg"})

In [21]:
min_val = 0.031500
max_val = 10

df_main_updated["sustainable footprint score"] = np.where(
    df_main_updated["Carbon-footprint 1kg"] > max_val, 0,
    ((max_val - df_main_updated["Carbon-footprint 1kg"]) / (max_val - min_val)) * 100).round(1)

df_main_updated.sample(2)

,Barcode,Product name,Brand,Quantity,Packaging material,Origin,Label,Sale countries,Product group,Food group,Product main category,Environmental score,environmental_score_grade,Carbon-footprint 1kg,sustainable footprint score
2194,7398818390165,Half 'N Half,Lohilo,175 g / 350 ml,NaN,NaN,No lactose,Finland,Ice cream,"Milk and dairy products,Ice cream",Ice creams,NaN,NaN,2.60,74.2
3312,4044889002577,Rice Choc 40% Cacao,Vivani,100 g e,"Paper,Tray",NaN,"Fair trade,Organic,Vegetarian,EU Organic,Fairt...","Finland,France",Chocolate products,"Sugary snacks,Chocolate products",Plant milk chocolates,NaN,unknown,2.95,70.7


In [22]:
df_main_updated.describe()

,Environmental score,Carbon-footprint 1kg,sustainable footprint score
count,927.000000,4603.000000,4603.000000
mean,55.625674,4.633615,67.891223
std,23.336007,8.091435,26.905102
min,-15.000000,0.031500,0.000000
25%,37.000000,2.050000,70.700000
50%,56.000000,2.650000,73.700000
75%,75.000000,2.950000,79.800000
max,114.000000,43.330000,100.000000


In [23]:
df_main_updated.isna().sum()

Barcode                           0
Product name                     38
Brand                           148
Quantity                       1045
Packaging material             2821
Origin                         3721
Label                          2132
Sale countries                    0
Product group                     0
Food group                        0
Product main category             0
Environmental score            3676
environmental_score_grade      3231
Carbon-footprint 1kg              0
sustainable footprint score       0
dtype: int64

In [24]:
df_main_updated['Product group'].value_counts()

Product group
Sweets                              340
One-dish meals                      292
Chocolate products                  234
Bread                               232
Biscuits and cakes                  229
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Milk and yogurt                     188
Yogurt                              188
Dairy desserts                      185
Breakfast cereals                   183
Protein snacks and drinks           158
Processed meat                      154
Cereals                             151
Ice cream                           131
Sweetened beverages                 118
Fats                                 78
Alcoholic beverages                  77
Poultry                              76
Vegetables                           70
Artificially sweetened beverages     68
Appetizers                           64
Pastries                             63
Canned foods              

In [25]:
df_main_updated["Product name"] = df_main_updated["Product name"].fillna('Product name unknown')
df_main_updated["Brand"] = df_main_updated["Brand"].fillna('Brand name unknown')
df_main_updated["Quantity"] = df_main_updated["Quantity"].fillna('Quantity unknown')
df_main_updated["Packaging material"] = df_main_updated["Packaging material"].fillna('Packaging unknown')
df_main_updated["Origin"] = df_main_updated["Origin"].fillna('Origin unknown')
df_main_updated["Label"] = df_main_updated["Label"].fillna('No information available')
df_main_updated["Environmental score"] = df_main_updated["Environmental score"].fillna('No information available')
df_main_updated["environmental_score_grade"] = df_main_updated["environmental_score_grade"].fillna('No information available')

In [26]:
df_main_updated.isna().sum()

Barcode                        0
Product name                   0
Brand                          0
Quantity                       0
Packaging material             0
Origin                         0
Label                          0
Sale countries                 0
Product group                  0
Food group                     0
Product main category          0
Environmental score            0
environmental_score_grade      0
Carbon-footprint 1kg           0
sustainable footprint score    0
dtype: int64

In [27]:
df_main_updated = df_main_updated.rename(columns={"environmental_score_grade": "Environmental score grade", 
                                                  "Carbon-footprint 1kg": "Carbon footprint 1kg",
                                                  "sustainable footprint score": "Sustainable footprint score"})
df_main_updated.sample(5)

,Barcode,Product name,Brand,Quantity,Packaging material,Origin,Label,Sale countries,Product group,Food group,Product main category,Environmental score,Environmental score grade,Carbon footprint 1kg,Sustainable footprint score
2229,6410381070150,Suklaa ja kaakaocreme,Brand name unknown,112 g,Packaging unknown,Origin unknown,"Sustainable farming,UTZ Certified",Finland,Biscuits and cakes,"Sugary snacks,Biscuits and cakes",Chocolate biscuits,No information available,No information available,2.50,75.2
3321,5000159512459,Twix Ginger cookie,"Twix,Mars",2 x 23g ℮,"Plastic,fi:green-dot",Origin unknown,Green Dot,"Finland,Italy",Chocolate products,"Sugary snacks,Biscuits and cakes",Bars-covered-with-chocolate,No information available,unknown,2.50,75.2
3201,6416453018778,Pure Dark - Twist of Mint,Fazer,95 g,"Metal,Recyclable Metals,Aluminium,Cardboard,No...",Origin unknown,"Vegetarian,Vegan,EAC,Green Dot","Denmark,Finland",Chocolate products,"Sugary snacks,Chocolate products",Dark chocolates,26.0,e,2.95,70.7
1911,6407870078797,Juusto makkara,Wilhelm,"4 kpl, 350 g",Packaging unknown,Origin unknown,"No gluten,No flavour enhancer,No lactose,No MS...",Finland,Processed meat,"Fish‚ Meat‚ Eggs,Processed meat",fi:juustomakkarat,No information available,No information available,13.15,0.0
2206,5600449992352,Dark chocolate with almond filling,Not-guilty,90 g,Packaging unknown,Origin unknown,"Vegetarian,Sustainable farming,Vegan,Green Dot...",Finland,Chocolate products,"Sugary snacks,Chocolate products",Filled dark chocolates,No information available,No information available,2.95,70.7


# FINAL DATABASE (Pipeline ready)

In [28]:
df_main_updated.to_csv("C:/code/repos/finalproject/dossier/donnees/final/sustainabite.csv")